In [2]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

tags_url = "https://www.instagram.com/explore/tags/"
my_tag = '냥이'
url = tags_url + my_tag

DRIVER_DIR = '/home/woomurf/dev/BigData Lab Project/instagramCrawl/chromedriver_linux64/chromedriver'
driver = webdriver.Chrome(DRIVER_DIR)
driver.implicitly_wait(3)

driver.get('chrome://settings/')
driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.25);')
driver.get(url)

elem = driver.find_element_by_tag_name("body")
link_list = []
num_of_crawling_pages = 10

pagedowns = 1
while pagedowns < num_of_crawling_pages:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        links = driver.find_elements_by_css_selector('div.v1Nh3 > a')
        for i in links:
            link_list.append(i.get_attribute('href'))
        pagedowns += 1

# 크롤링한 결과를 확인해 봅니다.
print('중복링크 개수', len(link_list))

set_link_list = list(set(link_list))
print('유니크링크 개수', len(set_link_list))

print('유니크링크 구성')
print(set_link_list[:10])

중복링크 개수 678
유니크링크 개수 519
유니크링크 구성
['https://www.instagram.com/p/BscUDSWHQyY/', 'https://www.instagram.com/p/BscNThAjDKb/', 'https://www.instagram.com/p/Bsca9VShckP/', 'https://www.instagram.com/p/BscI9T9Hbp9/', 'https://www.instagram.com/p/BscIsN6HQ04/', 'https://www.instagram.com/p/BscIe_BF0Z2/', 'https://www.instagram.com/p/BsccphFlmwo/', 'https://www.instagram.com/p/BscNRJLnk2l/', 'https://www.instagram.com/p/BscRJmBFp6s/', 'https://www.instagram.com/p/BscG5PXnb4A/']


In [21]:
first_url = 'https://www.instagram.com/p/Bsb5AXClIUy/'
print(first_url)

https://www.instagram.com/p/Bsb5AXClIUy/


In [22]:
import requests

req = requests.get(first_url)
html = req.text
header = req.headers
status = req.status_code
is_ok = req.ok

In [23]:
print(html)

<!DOCTYPE html>
<html lang="en" class="no-js not-logged-in client-root">
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

        <title>
🌼소봉소봉한 소봉이🌼 on Instagram: “소봉소봉한 소봉이
뭐야뭐야 누가 인형이야😻😻😻
.
.
.
.
.
.
#소봉소봉한#소봉이#인형#고양이#아메숏#아메숏믹스#예뻐라💕#내소봉이#집사#집사그램#냥이#냥스타그램#🐱#🐈#😻”
</title>

        
        <meta name="robots" content="noimageindex, noarchive">
        <meta name="mobile-web-app-capable" content="yes">
        <meta name="theme-color" content="#000000">
        <meta id="viewport" name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover">
        <link rel="manifest" href="/data/manifest.json">

        
          <link href="https://graph.instagram.com" rel="preconnect" crossorigin>
        

        <link rel="preload" href="/static/bundles/metro/PostPageContainer.js/afd525279fc3.js" as="script" type="text/javascript" crossorigin="anonymous" />
        

        <script t

In [24]:
print(header)

{'Content-Type': 'text/html; charset=utf-8', 'Vary': 'Accept-Language, Cookie, Accept-Encoding', 'Content-Language': 'en', 'Content-Encoding': 'gzip', 'Date': 'Thu, 10 Jan 2019 06:58:54 GMT', 'Strict-Transport-Security': 'max-age=3600', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src https: data: blob:; font-src https: data:; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninstagram.com wss://www.instagram.com https://*.facebook.com https://*.fbcdn.net https://*.facebook.net 'unsafe-inline' 'unsafe-eval' blob:; style-src 'self' https:

In [25]:
print(type(header))

<class 'requests.structures.CaseInsensitiveDict'>


In [26]:
print(is_ok)

True


In [27]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

## 처음에 찾아볼 때, 유저 아이디가 있는 a 태그의 FPmhX 클래스로 들어가려 했으나, JS 로 나중에 동기화 되어 찾을 수 없는듯 하다
# user_id = soup.find_all("a", class_ = "FPmhX") 의 user_id 의 len 은 0이다

## 하지만 다른 한 가지 방법으로 로드하기 이전 metadata 에 해당 게시물의 정보들이 들어있는 듯 하다.
## metadata 에는 본문이 짧게만 나오는 듯 하다
## 전체 본문 내용은 script 를 통해 출력되는 듯 하다. -> 하지만 인코딩이 다른 듯 하다
## display url 을 통해 사진의 링크도 출력된다.
## 결국 BS 로 추출해온 script content 에서 데이터를 뽑아내는게 핵심이다

script_content = soup.find_all("script")

print(script_content)

[<script type="text/javascript">
        (function() {
  var docElement = document.documentElement;
  var classRE = new RegExp('(^|\\s)no-js(\\s|$)');
  var className = docElement.className;
  docElement.className = className.replace(classRE, '$1js$2');
})();
</script>, <script type="text/javascript">
(function() {
  
  if ('PerformanceObserver' in window && 'PerformancePaintTiming' in window) {
    window.__bufferedPerformance = [];
    var ob = new PerformanceObserver(function(e) {
      window.__bufferedPerformance.push.apply(window.__bufferedPerformance,e.getEntries());
    });
    ob.observe({entryTypes:['paint']});
  }
  window.__bufferedErrors = [];
  window.onerror = function(message, url, line, column, error) {
    window.__bufferedErrors.push({
      message: message,
      url: url,
      line: line,
      column: column,
      error: error
    });
    return false;
  };
  window.__initialData = {
    pending: true,
    waiting: []
  };
  function notifyLoaded(item, data) {


In [28]:
meta_content = soup.find_all(property = "instapp:hashtags")
print(meta_content)


[<meta content="예뻐라💕" property="instapp:hashtags"/>, <meta content="🐈" property="instapp:hashtags"/>, <meta content="아메숏믹스" property="instapp:hashtags"/>, <meta content="🐱" property="instapp:hashtags"/>, <meta content="냥스타그램" property="instapp:hashtags"/>, <meta content="집사그램" property="instapp:hashtags"/>, <meta content="고양이" property="instapp:hashtags"/>, <meta content="인형" property="instapp:hashtags"/>, <meta content="아메숏" property="instapp:hashtags"/>, <meta content="소봉이" property="instapp:hashtags"/>, <meta content="냥이" property="instapp:hashtags"/>, <meta content="소봉소봉한" property="instapp:hashtags"/>, <meta content="내소봉이" property="instapp:hashtags"/>, <meta content="집사" property="instapp:hashtags"/>, <meta content="😻" property="instapp:hashtags"/>]


In [29]:
user_content = soup.find(rel="canonical")
username_pp = user_content.get('href')
print(username_pp)

https://www.instagram.com/sobonghan/p/Bsb5AXClIUy/


In [30]:
## 추출하고자 하는 내용들
## username ( 유저 아이디) -> OK
## contents ( 내용 ) -> OK
## tags ( 태그들 ) -> OK
## image_links ( 이미지 링크들 ) -> OK
## like_num ( 좋아요 개수 )
## comment_num ( 댓글 개수 )


import re


## 해당 script 전체 내용에서 username 의 내용만 추출하여 사용한다
username_p = re.compile("instagram.com/(.*)/p")
username = username_p.findall(str(username_pp))

## username이 작성자 뿐만 아니라 댓글을 단 사용자까지도 포함한다.
## 그런데 username이 웹에서 나타나는 댓글 순서 ( 작성자 : content, 댓글 : comment ) 순서가 아니라 섞여있어 username[0]이 무조건 작성자가 아니다.
print(username)



['sobonghan']


In [31]:
location_p = re.compile("location\":.*?\"name\":\"(.*?)\"")
location = location_p.findall(str(script_content))
if (len(location) > 0):
    location = location[0].encode('utf-8').decode('unicode_escape')
    print(location)
    

## unicode 된 것이 한글로만 이루어져 있으면 해당 방법을 통해 변환 가능하다.
## encode('utf-8').decode('unicode_escape')

In [65]:
try:
    import emoji
except:
    !pip install emoji
    import emoji

In [66]:
emoji.UNICODE_EMOJI

{'🥇': ':1st_place_medal:',
 '🥈': ':2nd_place_medal:',
 '🥉': ':3rd_place_medal:',
 '🆎': ':AB_button_(blood_type):',
 '🏧': ':ATM_sign:',
 '🅰': ':A_button_(blood_type):',
 '🇦🇫': ':Afghanistan:',
 '🇦🇱': ':Albania:',
 '🇩🇿': ':Algeria:',
 '🇦🇸': ':American_Samoa:',
 '🇦🇩': ':Andorra:',
 '🇦🇴': ':Angola:',
 '🇦🇮': ':Anguilla:',
 '🇦🇶': ':Antarctica:',
 '🇦🇬': ':Antigua_&_Barbuda:',
 '♒': ':Aquarius:',
 '🇦🇷': ':Argentina:',
 '♈': ':Aries:',
 '🇦🇲': ':Armenia:',
 '🇦🇼': ':Aruba:',
 '🇦🇨': ':Ascension_Island:',
 '🇦🇺': ':Australia:',
 '🇦🇹': ':Austria:',
 '🇦🇿': ':Azerbaijan:',
 '🔙': ':BACK_arrow:',
 '🅱': ':B_button_(blood_type):',
 '🇧🇸': ':Bahamas:',
 '🇧🇭': ':Bahrain:',
 '🇧🇩': ':Bangladesh:',
 '🇧🇧': ':Barbados:',
 '🇧🇾': ':Belarus:',
 '🇧🇪': ':Belgium:',
 '🇧🇿': ':Belize:',
 '🇧🇯': ':Benin:',
 '🇧🇲': ':Bermuda:',
 '🇧🇹': ':Bhutan:',
 '🇧🇴': ':Bolivia:',
 '🇧🇦': ':Bosnia_&_Herzegovina:',
 '🇧🇼': ':Botswana:',
 '🇧🇻': ':Bouvet_Island:',
 '🇧🇷': ':Brazil:',
 '🇮🇴': ':British_Indian_Ocean_Territory:',
 '🇻🇬': ':British_Vir

In [69]:
emoji = emoji.UNICODE_EMOJI.keys()
print(emoji)

dict_keys(['🥇', '🥈', '🥉', '🆎', '🏧', '🅰', '🇦🇫', '🇦🇱', '🇩🇿', '🇦🇸', '🇦🇩', '🇦🇴', '🇦🇮', '🇦🇶', '🇦🇬', '♒', '🇦🇷', '♈', '🇦🇲', '🇦🇼', '🇦🇨', '🇦🇺', '🇦🇹', '🇦🇿', '🔙', '🅱', '🇧🇸', '🇧🇭', '🇧🇩', '🇧🇧', '🇧🇾', '🇧🇪', '🇧🇿', '🇧🇯', '🇧🇲', '🇧🇹', '🇧🇴', '🇧🇦', '🇧🇼', '🇧🇻', '🇧🇷', '🇮🇴', '🇻🇬', '🇧🇳', '🇧🇬', '🇧🇫', '🇧🇮', '🆑', '🆒', '🇰🇭', '🇨🇲', '🇨🇦', '🇮🇨', '♋', '🇨🇻', '♑', '🇧🇶', '🇰🇾', '🇨🇫', '🇪🇦', '🇹🇩', '🇨🇱', '🇨🇳', '🇨🇽', '🎄', '🇨🇵', '🇨🇨', '🇨🇴', '🇰🇲', '🇨🇬', '🇨🇩', '🇨🇰', '🇨🇷', '🇭🇷', '🇨🇺', '🇨🇼', '🇨🇾', '🇨🇿', '🇨🇮', '🇩🇰', '🇩🇬', '🇩🇯', '🇩🇲', '🇩🇴', '🔚', '🇪🇨', '🇪🇬', '🇸🇻', '🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f', '🇬🇶', '🇪🇷', '🇪🇪', '🇪🇹', '🇪🇺', '🆓', '🇫🇰', '🇫🇴', '🇫🇯', '🇫🇮', '🇫🇷', '🇬🇫', '🇵🇫', '🇹🇫', '🇬🇦', '🇬🇲', '♊', '🇬🇪', '🇩🇪', '🇬🇭', '🇬🇮', '🇬🇷', '🇬🇱', '🇬🇩', '🇬🇵', '🇬🇺', '🇬🇹', '🇬🇬', '🇬🇳', '🇬🇼', '🇬🇾', '🇭🇹', '🇭🇲', '🇭🇳', '🇭🇰', '🇭🇺', '🆔', '🇮🇸', '🇮🇳', '🇮🇩', '🇮🇷', '🇮🇶', '🇮🇪', '🇮🇲', '🇮🇱', '🇮🇹', '🇯🇲', '🇯🇵', '🉑', '🈸', '🉐', '🏯', '㊗', '🈹', '🎎', '🈚', '🈁', '🈷', '🈵', '🈶', '🈺', '🈴', '🏣', '🈲', '🈯', '㊙', '🈂', '🔰', '🈳', '🇯🇪', '🇯🇴', '🇰🇿', '🇰🇪'

In [90]:
## meta data에서 태그들을 추출

hash_tags_p = re.compile("content=\"(.*?)\"")
hash_tags = hash_tags_p.findall(str(meta_content))

print(hash_tags)

# for e in emoji:
#     teee = tag.find(e)
#     print(teee)
#     print(type(e))
#     break

newHash_tags = []

# 태그에서 이모티콘을 제거한다.
for tag in hash_tags:
    for e in emoji:
        emoji_have = tag.find(e)
        
        if emoji_have > -1:
            tag = tag.replace(e,'')
    newHash_tags.append(tag)

newHash_tags = list(filter(None,newHash_tags))
print(newHash_tags)

['예뻐라💕', '🐈', '아메숏믹스', '🐱', '냥스타그램', '집사그램', '고양이', '인형', '아메숏', '소봉이', '냥이', '소봉소봉한', '내소봉이', '집사', '😻']
['예뻐라', '아메숏믹스', '냥스타그램', '집사그램', '고양이', '인형', '아메숏', '소봉이', '냥이', '소봉소봉한', '내소봉이', '집사']


In [94]:
## 이번에는 contents 추출하는 과정입니다
contents_p = re.compile("\"caption\":\"(.*?)\"")
contents = contents_p.findall(str(script_content))
caption = contents[0]

# caption = caption.replace("#"," ")

caption = caption.encode('utf-8')
caption = caption.decode('unicode_escape')

caption2 = caption.encode('utf-8','ignore')
caption2 = caption2.decode('utf-8')

print(caption2)


for tag in newHash_tags:
    tag = '#' + tag
    caption2 = caption2.replace(tag,"",1)

caption2 = caption2.replace('#','')

print()
print(caption2)


## 이 과정에서 태그가 함께 추출되므로, 이를 분리하는 과정이 약간 필요할 듯 합니다
## 단순하게 # 을 기준으로 태그를 추출할 경우, 본문에 # 을 썼을 경우 문제가 생길 수 있습니다. 
## 현재로서는 따로 태그가 저장되지는 않는 것으로 보입니다

## caption에서는 이모티콘을 제외했지만, 태그에서는 이모티콘이 그대로 남아있기 때문에 replace를 사용하여 지우는 것에 한계가 있습니다.
## 태그를 분리할 방법을 고안해야할 듯


소봉소봉한 소봉이
뭐야뭐야 누가 인형이야
.
.
.
.
.
.
#소봉소봉한#소봉이#인형#고양이#아메숏#아메숏믹스#예뻐라#내소봉이#집사#집사그램#냥이#냥스타그램###

소봉소봉한 소봉이
뭐야뭐야 누가 인형이야
.
.
.
.
.
.



In [16]:
# 작성 시간 추출

date_p = re.compile("uploadDate\":\"(.{19})\"")
date = date_p.findall(str(script_content))
print(date)

['2019-01-10T05:31:32']


In [17]:
## 이번에는 이미지 링크 추출 과정입니다
image_links_p = re.compile("\"display_resources\":\[.*?\]")
image_links = image_links_p.findall(str(script_content))
print(image_links)
print()

print(len(image_links))
print()

## 여러개의 이미지 링크들에서 개별 이미지 링크로 변환하는 과정입니다.
first_image_link_p = re.compile("\"src\":\".*?\"")
first_image_link = first_image_link_p.findall(str(image_links[0]))
print(first_image_link[0])
print(first_image_link[1])
print(first_image_link[2])
print(len(first_image_link))
print()

second_image_link_p = re.compile("\"src\":\".*?\"")
second_image_link = second_image_link_p.findall(str(image_links[1]))
print(second_image_link[0])
print(second_image_link[1])
print(second_image_link[2])
print(len(second_image_link))
print()

third_image_link_p = re.compile("\"src\":\".*?\"")
third_image_link = second_image_link_p.findall(str(image_links[2]))
print(third_image_link[0])
print(third_image_link[1])
print(third_image_link[2])
print(len(third_image_link))

## 이미지는 2개였지만, 총 3 종류의 링크가 존재했습니다
## 확인해 보니 3개중 1, 2번째 이미지는 같은 것이며 3번째는 다른 것이었습니다
## 추정해 보면 1번째 이미지는 대표 이미지, 2, 3번째 이미지가 포함된 이미지 2종류라고 생각해볼 수 있습니다
## 또한 각 이미지에 대해 resolution 에 의해 3가지의 다른 형태로 저장된 것을 볼 수 있습니다.
## 1번째는 640, 2번째는 750, 3번째는 원본 크기인것으로 보입니다

## 영상의 경우 이미지는 확인

['"display_resources":[{"src":"https://scontent-icn1-1.cdninstagram.com/vp/4bd7deeb109bc6826f98b2a8054d31d0/5CBA9238/t51.2885-15/sh0.08/e35/s640x640/47581626_326726064720220_6224719875477094754_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com","config_width":640,"config_height":592},{"src":"https://scontent-icn1-1.cdninstagram.com/vp/02b323dc38c4da37dbf203d16a44cb16/5CD60F38/t51.2885-15/sh0.08/e35/s750x750/47581626_326726064720220_6224719875477094754_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com","config_width":750,"config_height":694},{"src":"https://scontent-icn1-1.cdninstagram.com/vp/6315936e670e2e368c0509fbd246e30e/5CCB56DD/t51.2885-15/e35/47581626_326726064720220_6224719875477094754_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com","config_width":1080,"config_height":1000}]', '"display_resources":[{"src":"https://scontent-icn1-1.cdninstagram.com/vp/4bd7deeb109bc6826f98b2a8054d31d0/5CBA9238/t51.2885-15/sh0.08/e35/s640x640/47581626_326726064720220_6224719875477094754_n.jpg?_nc_ht=scontent

In [18]:
## 이번에는 like_num, comment_num 추출하는 과정입니다
likes_comments_num_p = re.compile("\"description\":\".*?Comments")
likes_comments_num = likes_comments_num_p.findall(str(script_content))
print(likes_comments_num)
print()

['"description":"11 Likes, 0 Comments']



In [19]:
## 이제 해당 내용을 정리하여 JSON 객체로 저장하면 됩니다.

## 한 것들
### username 
### tags
### image
### date
### Location

## 아직 못한 것들
### description ( caption )
### caption에서 태그분리

